Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [32]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [34]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON, DelimitedFiles
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

using AmbulanceDeployment, Dates

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`


In [36]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [49]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)#Distributions.LogNormal(3.089, 0.3) #Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
#include("..//src//evaluate.jl") #if you include, you need to include JuMP
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls_v1.csv")|> DataFrame 
#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [51]:
test_calls

,interarrival_seconds,neighborhood,dow,stn1_min,stn2_min,stn3_min,stn4_min
,Int64,Int64,String,Float64,Float64,Float64,Float64
1,38,83,Wed,5.0,26.9957,10.791,13.1813
2,537,38,Wed,16.873,26.3413,22.8293,22.9943
3,98,129,Wed,21.5312,38.0153,22.2628,18.438
4,18,107,Wed,44.2525,37.6413,36.6008,50.837
5,475,113,Wed,14.653,29.5155,15.3847,13.4282
6,21,51,Wed,21.06,13.8817,18.8858,31.445
7,263,76,Wed,43.5517,34.0775,36.4807,52.2222
8,128,67,Wed,14.8368,17.8243,9.6105,23.073
9,62,52,Wed,17.9115,22.8192,17.5257,27.8428


In [53]:
problem.emergency_calls[:, Symbol("stn$(1)_min")]

1000-element Array{Float64,1}:
  5.0
 16.873
 21.531166666666664
 44.252500000000005
 14.652999999999999
 21.06
 43.55166666666666
 14.836833333333335
 17.9115
 14.836833333333335
 14.836833333333335
 10.3225
 10.132
  ⋮
 20.229333333333333
 19.207
  5.0
 20.229333333333333
 20.229333333333333
 14.652999999999999
 14.080333333333334
 14.652999999999999
 12.603666666666667
 20.305333333333333
 14.080333333333334
 10.3225

In [55]:
function gridTime2Bin(gridTime)
    weights = [201.70483660130716, 10.166325878594247, -208.83953125000002, -460.87083333333334,-775.0755555555556,-1046.168888888889,-1527.7117647058826]
    bins = [ 250,  550,  850, 1150, 1450, 1750]
    for i in 1:length(bins)
        if(gridTime < bins[i])
            return (i)
        end
    return length(bins) + 1
    end
end

gridTime2Bin (generic function with 1 method)

In [62]:
"""Simulation Engine"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        travel_time = travel_time -146 #+ gridTime2Bin(travel_time) #adjust time
        #travel_time = ceil(travel_time * .9) #time mod
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    #conveytime = ceil(conveytime * .9) #time mod
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    #returntime = ceil(returntime * .9) #time mod
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [63]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
Ambulance has arrived on the scene for event 1 at time 192
calling event id: 2 time: 575 value: 38
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
Ambulance for event 1 has arrived at the hospital at time 875
Ambulance has arrived on the scene for event 2 at time 1104
Ambulance has arrived on the scene for event 4 at time 1156
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
Ambulance has arrived on the scene for event 3 at time 1261
Ambulance has arrived on the scene for event 6 at time 1341
calling event id: 7 time: 1450 value: 76
calling event id: 8 time: 1578 value: 67
Ambulance has arrived on the scene for event 5 at time 1606
calling event id: 9 time: 1640 value: 52
Ambulance has arrived on the scene for event 8 at time 1732
calling event id: 10 time: 1838 value: 67
Ambulance for event 4 has arrived at the hospital at time 1892
calling event id: 11 time: 1899 value: 67
calli

Ambulance has arrived on the scene for event 34 at time 9823
Ambulance for event 33 has arrived at the hospital at time 9874
Ambulance has arrived on the scene for event 36 at time 10006
calling event id: 37 time: 10091 value: 98
Ambulance for event 29 is returning to the station at 10125
Ambulance from event 21 has returned to station 43
Ambulance for event 27 is returning to the station at 10317
Ambulance for event 32 has arrived at the hospital at time 10404
Ambulance for event 34 has arrived at the hospital at time 10494
Ambulance from event 27 has returned to station 17
Ambulance from event 29 has returned to station 3
Ambulance for event 35 has arrived at the hospital at time 10543
Ambulance has arrived on the scene for event 37 at time 10553
calling event id: 38 time: 10595 value: 98
Ambulance for event 30 is returning to the station at 10657
Ambulance for event 31 is returning to the station at 10686
calling event id: 39 time: 10727 value: 52
calling event id: 40 time: 10770 va

Ambulance from event 55 has returned to station 17
calling event id: 63 time: 19271 value: 98
Ambulance for event 60 is returning to the station at 19386
calling event id: 64 time: 19691 value: 51
Ambulance has arrived on the scene for event 63 at time 19733
Ambulance has arrived on the scene for event 64 at time 19845
Ambulance for event 62 has arrived at the hospital at time 19867
calling event id: 65 time: 19963 value: 98
Ambulance for event 59 is returning to the station at 19969
Ambulance from event 60 has returned to station 1
Ambulance for event 61 has arrived at the hospital at time 20238
calling event id: 66 time: 20471 value: 113
Ambulance from event 59 has returned to station 14
Ambulance has arrived on the scene for event 65 at time 20639
calling event id: 67 time: 20814 value: 36
Ambulance for event 64 has arrived at the hospital at time 20831
Ambulance for event 63 has arrived at the hospital at time 20909
Ambulance has arrived on the scene for event 66 at time 20911
call

Ambulance for event 85 is returning to the station at 31244
Ambulance for event 89 is returning to the station at 31826
Ambulance for event 92 is returning to the station at 32057
Ambulance from event 87 has returned to station 22
Ambulance from event 85 has returned to station 22
Ambulance from event 89 has returned to station 25
calling event id: 93 time: 32356 value: 66
Ambulance for event 91 is returning to the station at 32422
Ambulance has arrived on the scene for event 93 at time 32510
Ambulance from event 91 has returned to station 8
Ambulance from event 92 has returned to station 14
calling event id: 94 time: 32957 value: 66
Ambulance for event 90 is returning to the station at 33079
Ambulance has arrived on the scene for event 94 at time 33111
Ambulance from event 90 has returned to station 39
Ambulance for event 93 has arrived at the hospital at time 33418
calling event id: 95 time: 33904 value: 99
calling event id: 96 time: 33986 value: 142
Ambulance has arrived on the scen

Ambulance from event 113 has returned to station 14
calling event id: 126 time: 45420 value: 143
Ambulance for event 116 has arrived at the hospital at time 45430
Ambulance for event 117 has arrived at the hospital at time 45536
Ambulance has arrived on the scene for event 124 at time 45708
calling event id: 127 time: 45780 value: 97
Ambulance has arrived on the scene for event 126 at time 45830
Ambulance for event 122 has arrived at the hospital at time 45880
Ambulance for event 120 has arrived at the hospital at time 45904
Ambulance for event 115 is returning to the station at 45926
Ambulance has arrived on the scene for event 127 at time 46008
Ambulance from event 115 has returned to station 8
calling event id: 128 time: 46110 value: 82
Ambulance has arrived on the scene for event 125 at time 46143
Ambulance for event 121 has arrived at the hospital at time 46232
Ambulance for event 123 has arrived at the hospital at time 46491
calling event id: 129 time: 46504 value: 141
calling ev

...

Ambulance from event 899 has returned to station 8
Ambulance for event 903 is returning to the station at 255535
Ambulance has arrived on the scene for event 910 at time 255553
Ambulance has arrived on the scene for event 912 at time 255563
Ambulance for event 904 is returning to the station at 255610
Ambulance for event 901 is returning to the station at 255763
Ambulance for event 896 is returning to the station at 255776
Ambulance from event 894 has returned to station 4
Ambulance has arrived on the scene for event 911 at time 255862
Ambulance from event 904 has returned to station 5
Ambulance from event 896 has returned to station 8
Ambulance for event 898 is returning to the station at 255944
Ambulance from event 903 has returned to station 11
Ambulance from event 897 has returned to station 22
Ambulance for event 910 has arrived at the hospital at time 256164
Ambulance for event 906 is returning to the station at 256199
calling event id: 913 time: 256204 value: 112
calling event i

Ambulance from event 928 has returned to station 5
calling event id: 937 time: 261778 value: 129
937: call from 129 is a shortfall
Ambulance has arrived on the scene for event 939 at time 261804
Ambulance for event 923 is returning to the station at 261811
calling event id: 941 time: 261834 value: 99
941: call from 99 is a shortfall
Ambulance from event 926 has returned to station 9
calling event id: 937 time: 262004 value: 129
937: call from 129 is a shortfall
calling event id: 941 time: 262004 value: 99
941: call from 99 is a shortfall
calling event id: 942 time: 262019 value: 51
Ambulance from event 920 has returned to station 19
calling event id: 937 time: 262028 value: 129
937: call from 129 is a shortfall
calling event id: 941 time: 262028 value: 99
941: call from 99 is a shortfall
Ambulance for event 925 is returning to the station at 262038
Ambulance for event 927 is returning to the station at 262119
Ambulance has arrived on the scene for event 942 at time 262173
Ambulance has

Ambulance from event 947 has returned to station 17
calling event id: 959 time: 265917 value: 82
Ambulance for event 949 is returning to the station at 266007
Ambulance for event 950 is returning to the station at 266046
calling event id: 960 time: 266062 value: 68
Ambulance for event 948 is returning to the station at 266124
Ambulance for event 953 is returning to the station at 266152
Ambulance for event 955 has arrived at the hospital at time 266157
Ambulance has arrived on the scene for event 960 at time 266216
Ambulance for event 951 is returning to the station at 266304
Ambulance for event 941 is returning to the station at 266323
Ambulance for event 937 is returning to the station at 266373
Ambulance has arrived on the scene for event 959 at time 266391
Ambulance from event 950 has returned to station 17
calling event id: 961 time: 266455 value: 142
Ambulance from event 951 has returned to station 8
Ambulance from event 948 has returned to station 3
calling event id: 962 time: 2

Ambulance from event 975 has returned to station 9
Ambulance for event 982 has arrived at the hospital at time 274924
Ambulance for event 976 is returning to the station at 275174
Ambulance has arrived on the scene for event 983 at time 275186
calling event id: 984 time: 275597 value: 98
calling event id: 985 time: 275709 value: 51
calling event id: 986 time: 275745 value: 98
Ambulance from event 976 has returned to station 9
Ambulance for event 983 has arrived at the hospital at time 275847
Ambulance has arrived on the scene for event 985 at time 275863
Ambulance for event 978 is returning to the station at 275865
Ambulance for event 980 is returning to the station at 275874
Ambulance from event 978 has returned to station 8
Ambulance has arrived on the scene for event 984 at time 276059
Ambulance has arrived on the scene for event 986 at time 276207
Ambulance for event 977 is returning to the station at 276224
Ambulance for event 979 is returning to the station at 276293
Ambulance fr

(1000×11 DataFrame. Omitted printing of 6 columns
│ Row  │ id    │ dispatch_from │ waittime │ responsetime │ scenetime │
│      │ Int64 │ Int64         │ Float64  │ Float64      │ Float64   │
├──────┼───────┼───────────────┼──────────┼──────────────┼───────────┤
│ 1    │ 1     │ 1             │ 0.0      │ 2.56667      │ 11.3833   │
│ 2    │ 2     │ 43            │ 0.0      │ 8.81667      │ 13.35     │
│ 3    │ 3     │ 14            │ 0.0      │ 9.8          │ 13.0      │
│ 4    │ 4     │ 34            │ 0.0      │ 7.75         │ 12.2667   │
│ 5    │ 5     │ 17            │ 0.0      │ 7.33333      │ 11.25     │
│ 6    │ 6     │ 22            │ 0.0      │ 2.56667      │ 16.25     │
│ 7    │ 7     │ 39            │ 0.0      │ 22.1167      │ 12.1333   │
│ 8    │ 8     │ 8             │ 0.0      │ 2.56667      │ 19.8333   │
│ 9    │ 9     │ 8             │ 0.0      │ 8.91667      │ 12.75     │
│ 10   │ 10    │ 3             │ 0.0      │ 7.18333      │ 9.08333   │
⋮
│ 990  │ 990   │ 44      

In [ ]:
"""batch call simulation"""
# batches = 12
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# using DelimitedFiles

# for i = 1:12
#     """ simulation parameters """
#     p = DeploymentProblem(
#               hourly_calls,
#               adjacent_nbhd,
#               coverage,
#               namb = namb,
#               train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#               )
#     test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#     #update 1: remove negative indices
#     start_ind = (i-1)*1000+1
#     end_ind = (i)*1000
#     println("running $namb ambuances & $start_ind to $end_ind calls")
#     test_calls_batch = test_calls[start_ind:end_ind,:]

#     # distribution of ambulances across all of the stations
#     x = amb_deployment[model_dict[model_name]][namb]

#     #establishes the dispatch problem
#     problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

#     #establishes the dispatch model
#     dispatch = ClosestDispatch(p, problem)

#     #@time df, guiArray = simulate_events!(problem, dispatch)
#     df, guiArray = simulate_events!(problem, dispatch)
#     print(mean(df["responsetime"]))
    
#     filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
#     open(filename, "w") do filename
#         writedlm(filename,  df["responsetime"])
#     end
#     print(filename)
# end

In [ ]:
"""all models simulation"""
# turnaround = Distributions.LogNormal(3.65, 0.3)
# lambda = 0
# local_path = ""
# model_name = "Stochastic"
# namb = 40
# ncalls = 1000
# #using AmbulanceDeployment

# pwd()

# cd(currentpath * "/test/")
# pwd()

# #using AmbulanceDeployment
# using Dates

# #include("..//src//dispatch/closestdispatch.jl")
# include("..//src//problem.jl")
# #include("..//src//evaluate.jl") #if you include, you need to include JuMP
# #include("..//src//simulate.jl")

# hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
# coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
# coverage = convert(Array{Bool, 2}, coverage[:, :])
# incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
# solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
# amb_deployment = solverstats["amb_deployment"]
# model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
#           "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


# p = DeploymentProblem(
#           hourly_calls,
#           adjacent_nbhd,
#           coverage,
#           namb = namb,
#           train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#           )
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# test_calls = test_calls[1:ncalls,:]
# println("running $namb ambuances & $ncalls calls")

# # distribution of ambulances across all of the stations
# x = amb_deployment[model_dict[model_name]][namb]

# #establishes the dispatch problem
# problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

# #establishes the dispatch model
# dispatch = ClosestDispatch(p, problem)

# Random.seed!(1234)

In [65]:
println(mean(df["responsetime"]))
println(mean(df["scenetime"]))
println(mean(df["conveytime"]))
println(mean(df["returntime"]))

7.7477
16.250616666666666
24.4176
9.114916666666668


println(mean(df["responsetime"]))
println(mean(df["scenetime"]))
println(mean(df["conveytime"]))
println(mean(df["returntime"]))
#### lognromal 3.08
9.813166666666667 <br>
9.134116666666667 <br>
24.4176 <br>
8.908 <br>
#### lognromal 3.65
10.3443 <br>
16.000433333333337 <br>
24.4176 <br>
9.090300000000001 <br>
#### adjust travel time by time_bin weights
10.466350000000002 <br>
16.000433333333334 <br>
24.4176 <br>
9.073 <br>
#### global
7.7477 <br>
16.250616666666666 <br>
24.4176 <br>
9.114916666666668 <br>


In [ ]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

In [ ]:
using DelimitedFiles
filename = string("visualization/time_adjust/",model_name,string(namb),"calls",string(ncalls),"stochastic40_timebins",".csv")
open(filename, "w") do filename
    writedlm(filename,  df["responsetime"])
end
print(filename)

In [ ]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")